# Sales Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

In [2]:
df = pd.read_csv('Sales_Data/Sales_April_2019.csv')
df.head(3)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"


### Merge multiple CSV files into a single one

In [3]:
files = [file for file in os.listdir('Sales_Data/')]
for file in files:
    print(file)

Sales_July_2019.csv
Sales_October_2019.csv
Sales_December_2019.csv
Sales_April_2019.csv
Sales_January_2019.csv
Sales_June_2019.csv
Sales_February_2019.csv
Sales_November_2019.csv
Sales_September_2019.csv
Sales_March_2019.csv
Sales_August_2019.csv
Sales_May_2019.csv


In [ ]:
all_months_data = pd.DataFrame()

for file in files:
    df = pd.read_csv('Sales_Data/'+file)
    all_months_data = pd.concat([all_months_data,df])
    
all_months_data.to_csv('all_data.csv', index=False)

### Read in updated dataframe

In [ ]:
all_data = pd.read_csv('all_data.csv')
all_data.head(3)

### Clear Data

In [ ]:
all_data['Month'] = all_data['Order Date'].str[:2]
all_data['Year'] = all_data['Order Date'].str[6:8]
all_data.head(3)

### Drop rows of NaN

In [ ]:
all_data[all_data.isna().any(axis=1)].head(3)

In [ ]:
# Drop all columns with NaN values in every fields
all_data = all_data.dropna(how='all') #'any', Drops the entire line when appears at least one NaN value

In [ ]:
all_data.groupby('Month').count() # We have an indesirable 'Or' in Month column. Lest delete it.

In [ ]:
temp_df = all_data[all_data['Order Date'].str[:2] == 'Or']  # Show all 'Or values'
temp_df

In [ ]:
all_data = all_data[all_data['Order Date'].str[:2] != 'Or'] # Delete every column with 'Or' value 

In [ ]:
all_data['Month'] = all_data['Month'].astype('int32') # Converting Month into an int32

In [ ]:
all_data.head(3)

In [ ]:
all_data.info()

In [ ]:
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered']) # Mmake int
all_data['Price Each'] = pd.to_numeric(all_data['Price Each']) # Mmake float
all_data.info()

### Add a Total Price Column

In [ ]:
all_data['Total Price'] = all_data['Quantity Ordered'] * all_data['Price Each']
all_data.head(3)

### Q1: What was the best sales month?

In [ ]:
sales = all_data.groupby('Month').sum()
sales

In [ ]:
fig1, (ax1, ax2) = plt.subplots(1, 2, sharey=False, figsize=(16,4))

x = np.arange(1,13,1)

plt.style.use('ggplot')
fig1.suptitle('Sales per Month', size=30)

ax1.bar(x, sales['Total Price'], color='#1c4bba')

ax2.plot(x, sales['Total Price'], 'b--o')

plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
months = range(1, 13)
y = sales['Total Price']

plt.bar(months, y, color='#329c78')
# plt.plot(months, sales['Total Price'], 'r.-')
plt.title('Sales Per Month', fontsize=25)
plt.xticks(months)
plt.xlabel('Months')
plt.ylabel('Amount')

for i in months:
    plt.text(i, y[i], round(y[i], 2), ha="center", va="top", rotation="vertical", color="white")

plt.show()

### Q2:  What city had the highest number of sales?

In [ ]:
all_data.head(3)

### Add a City column

In [ ]:
def get_city(address):
    return address.split(',')[1]

def get_state(address):
    return address.split(',')[2].split(' ')[1]

all_data['City'] = all_data['Purchase Address'].apply(lambda x: get_city(x) + '-' + get_state(x))

# all_data['City'] = all_data['Purchase Address'].apply(lambda x: x.split(',')[1])
all_data.head(3)

In [ ]:
top_city = all_data.groupby('City').sum()
top_city

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

x = all_data['City'].unique()
y = top_city['Total Price']

plt.barh(x, y.sort_values(ascending=False))
plt.title('Sales by City for Sales')
plt.xticks(rotation=-10)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
# palette = sns.color_palette("Blues", 10)
palette = sns.cubehelix_palette(10, start=2, rot=0, dark=0.1, light=.80, reverse=True)
x = all_data['City'].unique()
y = top_city['Total Price'].sort_values(ascending=False)

sns.barplot(data=top_city, x=x, y=y, palette=palette)
plt.title('Sales Per City')
plt.xticks(rotation=45)
plt.ylim(0, 10000000)

for i in range(len(x)):
    y[i] = y[i].astype("int32")
    plt.text(i, y[i], y[i], ha="center", va="bottom", rotation="vertical", color="darkgreen")

plt.show()

### Q3: When is the best time to ads?

### Convert "Order Date" in a DateTime object

In [ ]:
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])

In [ ]:
all_data['Hour'] = all_data['Order Date'].dt.hour # Creates the Hour field
all_data['Minute'] = all_data['Order Date'].dt.minute
all_data.head(1)

In [ ]:
best_hour = all_data.groupby('Hour').sum()
best_hour.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

hours = [hour for hour, df in all_data.groupby('Hour')]
y = all_data.groupby(['Hour']).count()

plt.plot(hours, y, color="Blue")
plt.title("Best Time To Advertisements", fontsize=25)
plt.xticks(hours)
plt.axhline(y=12000, xmin=0.1, xmax=0.9)

plt.show()

### Q4: Which products are often sold together?

In [ ]:
df = all_data[all_data['Order ID'].duplicated(keep=False)]

#Join product names at the same order
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

#Delete duplicates
df = df[['Order ID', 'Grouped']].drop_duplicates()

df.head()

In [ ]:
#Reference: https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary
from itertools import combinations
from collections import  Counter

count = Counter()

for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))
    
for key, value in count.most_common(10):
    print(key, value)

### Q5: What product are mostly sold and why do you think it is sold the most?

In [ ]:
all_data.head(3)

In [ ]:
product_group = all_data.groupby('Product')
quantity_ordered = product_group['Quantity Ordered'].sum()

products = [product for product, df in product_group]

fig, ax = plt.subplots(figsize=(10,4))

plt.bar(products, quantity_ordered.sort_values(ascending=False), color="darkgreen")
plt.title("Most Sold Product", fontsize=25)
plt.xticks(rotation='vertical')
plt.ylabel("Quantity Ordered")

plt.show()

In [ ]:
# product_group = all_data.groupby('Product')
# quantity_ordered = product_group['Quantity Ordered'].sum()
prices = all_data.groupby('Product')['Price Each'].mean()

# products = [product for product, df in product_group]

fig, ax1 = plt.subplots(figsize=(10,4))

ax2 = ax1.twinx()
ax1.bar(products, quantity_ordered.sort_values(ascending=False))
ax2.plot(products, prices, 'g.-')

fig.suptitle("Most Sold Product / Their price", fontsize=25)
ax1.set_xticklabels(products, rotation='vertical')
ax1.set_ylabel("Quantity Ordered")
ax2.set_ylabel("Price")

plt.show()